# Function calling

In this example, we will use function calling (tools) to obtain information from a Git repository.

## Dependencies

We will use LangChain again and we also need GitPython to interact with a Git repository.

In [ ]:
import sys
!{sys.executable} -m pip install GitPython langchain-openai langchain-core

## Model

We will use an OpenAI model again, so we need an API key.

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [ ]:
!git clone https://github.com/rjust/defects4j.git

## Defining the functions

Let's define the functions (tools) that can be called to fulfill a task sent to the model.

It is important to give informative names for the function and the parameters and provide some form of documentation. These details will be provided in the context of the query to the model.

In [ ]:
from datetime import date
from typing import List
from langchain_core.tools import tool
from git import Git

repo_directory = './defects4j'

@tool
def list_commits_between(start_date: date, end_date: date) -> str:
    """ Returns the messages of the commits from an interval between dates.
    Args:
        start_date: The start of the interval.
        end_date: The end of the date.
    """
    repo = Git(repo_directory)
    logs = repo.log('--since={}'.format(start_date), '--until={}'.format(end_date), '--oneline')
    return logs

@tool
def list_commits_from(contributor: str) -> str:
  """ Returns the commits from a specific contributor.
  Args:
      contributor: The name of the contributor.
  """
  repo = Git(repo_directory)
  logs = repo.log('--author={}'.format(contributor))
  return logs

## Binding the functions to the model

Now, let's register the functions into the model. After that, the calls to the model will inform about the existence of the functions, what they can do and their parameters.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools([list_commits_between, list_commits_from])

## Querying the model

Let's ask the model to summarize what was done in the project in the last year.

To do so, we have to tell the model the date of today (do you want to try without doing so?)).

In [ ]:
from datetime import date

query = '''Summarize what has been done in the project last calendar year. Today is {}.
          In your answer, provide the dates of which your results refer to.'''.format(date.today())

Now let's query the model:

In [ ]:
msg_with_tool_calls = llm_with_tools.invoke(query)

Let's inspect the response.

In [ ]:
msg_with_tool_calls.pretty_print()

Now, we have to pass the previous messages to the model. Let's prepare the list of messages.

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query), msg_with_tool_calls]

for tool_call in msg_with_tool_calls.tool_calls:
    selected_tool = locals()[tool_call['name']]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    tool_msg.pretty_print()

Now, it is time to send to the model and see the answer.

In [ ]:
result = llm_with_tools.invoke(messages)
result.pretty_print()

## Exercise

1. Modify the code to ask about a specific contributor.